In [10]:
pip install pycaret

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.9/485.9 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 44.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.5/160.5 kB 14.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.0/258.0 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.8/106.8 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.7/80.7 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.9/21.9 MB 46.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 64.8 M

In [1]:
import pandas as pd

In [2]:
url = 'https://raw.githubusercontent.com/PedroCosDi/p4dataedgeflow/main/queimadas.csv'
df = pd.read_csv(url, encoding='ISO-8859-1', sep=';')

In [3]:
print(df.head())

      date                     class  focuses           uf
0  2024/01  Desmatamento Consolidado        3         ACRE
1  2024/01  Desmatamento Consolidado        1        AMAPÁ
2  2024/01  Desmatamento Consolidado       22     AMAZONAS
3  2024/01  Desmatamento Consolidado       69     MARANHÃO
4  2024/01  Desmatamento Consolidado      274  MATO GROSSO


In [4]:
#Convertendo a coluna date em datetime e colocando como index
df['date'] = pd.to_datetime(df['date'])
df.set_index('date', inplace=True)
# Ordenar o DataFrame pelo índice (data)
df.sort_index(inplace=True)


<ipython-input-4-3880866dc36f>:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['date'] = pd.to_datetime(df['date'])


In [5]:
print(df.head())

                           class  focuses           uf
date                                                  
2019-08-01  Vegetacao Secundaria      616         PARÁ
2019-08-01              Floresta      454  MATO GROSSO
2019-08-01              Floresta      696         PARÁ
2019-08-01              Floresta      274     RONDÔNIA
2019-08-01              Floresta        2      RORAIMA


In [6]:
# Remover a coluna 'class' do DataFrame completo
df_sem_class = df.drop(columns=['class'])

# Agrupar o DataFrame sem a coluna 'class' por estado e data, somando os valores da coluna 'focuses'
df_agrupado = df_sem_class.groupby(['date', 'uf']).sum()

# Resetar o índice para manter 'date' como o índice
df_agrupado.reset_index(inplace=True)

# Classificar os valores por data
df_agrupado.sort_values(by='date', inplace=True)

# Definir 'date' como índice
df_agrupado.set_index('date', inplace=True)

# Agrupar novamente por estado, desta vez agregando as linhas com o mesmo estado
grupos_por_estado = df_agrupado.groupby('uf')

# Criar um dicionário para armazenar os DataFrames separados por estado
dataframes_por_estado = {}

# Iterar sobre os grupos e criar um DataFrame separado para cada estado
for estado, grupo in grupos_por_estado:
    # Remover a coluna 'uf'
    grupo.drop(columns=['uf'], inplace=True)

    dataframes_por_estado[estado] = grupo

df_acre = dataframes_por_estado['ACRE']
df_amapa = dataframes_por_estado['AMAPÁ']
df_amazonas = dataframes_por_estado['AMAZONAS']
df_mato_grosso = dataframes_por_estado['MATO GROSSO']
df_rondonia = dataframes_por_estado['RONDÔNIA']
df_roraima = dataframes_por_estado['RORAIMA']
df_maranhao = dataframes_por_estado['MARANHÃO']
df_tocantins = dataframes_por_estado['TOCANTINS']
df_para = dataframes_por_estado['PARÁ']

In [7]:
print(df_amazonas.head())

            focuses
date               
2019-08-01     6668
2019-09-01     3026
2019-10-01      548
2019-11-01      573
2019-12-01      163


In [8]:
# Resetar o índice para manter 'date' como uma coluna regular
df_amazonas.reset_index(inplace=True)

# Converter a coluna 'date' para o formato de período mensal
df_amazonas['date'] = pd.to_datetime(df_amazonas['date']).dt.to_period('M')

# Definir a coluna 'date' como índice novamente
df_amazonas.set_index('date', inplace=True)

# Verificar o DataFrame resultante
print(df_amazonas.head())

         focuses
date            
2019-08     6668
2019-09     3026
2019-10      548
2019-11      573
2019-12      163


In [11]:
from pycaret.time_series import *

In [12]:
print(df_amazonas.index)

PeriodIndex(['2019-08', '2019-09', '2019-10', '2019-11', '2019-12', '2020-01',
             '2020-02', '2020-03', '2020-04', '2020-05', '2020-06', '2020-07',
             '2020-08', '2020-09', '2020-10', '2020-11', '2020-12', '2021-01',
             '2021-02', '2021-03', '2021-04', '2021-05', '2021-06', '2021-07',
             '2021-08', '2021-09', '2021-10', '2021-11', '2021-12', '2022-01',
             '2022-02', '2022-03', '2022-04', '2022-05', '2022-06', '2022-07',
             '2022-08', '2022-09', '2022-10', '2022-11', '2022-12', '2023-01',
             '2023-02', '2023-03', '2023-04', '2023-05', '2023-06', '2023-07',
             '2023-08', '2023-09', '2023-10', '2023-11', '2023-12', '2024-01'],
            dtype='period[M]', name='date')


In [13]:
print(df_amazonas.info())

<class 'pandas.core.frame.DataFrame'>
PeriodIndex: 54 entries, 2019-08 to 2024-01
Freq: M
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype
---  ------   --------------  -----
 0   focuses  54 non-null     int64
dtypes: int64(1)
memory usage: 864.0 bytes
None


In [14]:
# Chamar o método setup()
setup(df_amazonas, fh=6, fold=3)

,Description,Value
0,session_id,2402
1,Target,focuses
2,Approach,Univariate
3,Exogenous Variables,Not Present
4,Original data shape,"(54, 1)"
5,Transformed data shape,"(54, 1)"
6,Transformed train set shape,"(48, 1)"
7,Transformed test set shape,"(6, 1)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


In [15]:
# Escolher os melhores modelos
best_model = compare_models()

,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2,TT (Sec)
theta,Theta Forecaster,1.2547,1.4834,445.3112,896.9852,0.7361,0.5527,0.6756,0.0700
snaive,Seasonal Naive Forecaster,1.3458,1.5156,465.7222,890.0033,0.5009,0.5168,0.7969,0.1033
ets,ETS,1.4460,1.5283,521.7792,985.8117,0.5756,0.5130,-0.0897,0.4433
omp_cds_dt,Orthogonal Matching Pursuit w/ Cond. Deseasonalize & Detrending,1.4509,1.4763,560.9849,952.1801,5.6157,0.9508,0.0232,0.3767
arima,ARIMA,1.4965,1.5407,540.5804,931.7710,3.4866,0.9302,0.6508,0.1767
xgboost_cds_dt,Extreme Gradient Boosting w/ Cond. Deseasonalize & Detrending,1.5033,1.4538,554.7835,912.7160,4.7569,0.9785,0.3892,0.5767
exp_smooth,Exponential Smoothing,1.5764,1.6185,556.4028,982.1674,1.4381,0.7476,0.5758,0.1467
br_cds_dt,Bayesian Ridge w/ Cond. Deseasonalize & Detrending,1.6363,1.6597,641.8328,1071.2170,7.4972,1.2800,-0.2372,0.3000
et_cds_dt,Extra Trees w/ Cond. Deseasonalize & Detrending,1.7003,1.6432,671.7093,1101.2161,7.9730,1.1595,-0.9383,0.5000
stlf,STLF,1.7443,1.5679,674.2226,1013.8655,8.1155,1.2437,-0.1051,0.0867


Processing:   0%|          | 0/117 [00:00<?, ?it/s]

In [16]:
# Criar o modelo Theta
theta = create_model('theta')

,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,R2
0,2022-01,0.2705,0.3302,116.2475,215.4999,1.2056,0.6609,0.8264
1,2022-07,3.2352,3.9022,1085.4106,2214.9637,0.3635,0.4891,0.6355
2,2023-01,0.2583,0.2178,134.2756,260.4919,0.6393,0.5082,0.5648
Mean,NaT,1.2547,1.4834,445.3112,896.9852,0.7361,0.5527,0.6756
SD,NaT,1.4005,1.7109,452.6784,932.1325,0.3505,0.0769,0.1105


Processing:   0%|          | 0/4 [00:00<?, ?it/s]

In [17]:
# Visualizar decomposição
plot_model(theta, plot='decomp_stl')

In [18]:
#Visualizar dados de treino e teste
plot_model(theta, plot='train_test_split')

In [19]:
# Finaliza o treinamento do modelo
final = finalize_model(theta)

In [20]:
final

ForecastingPipeline(steps=[('forecaster',
                            TransformedTargetForecaster(steps=[('model',
                                                                ThetaForecaster(sp=12))]))])

In [21]:
# Visualização da previsão
plot_model(final, plot='forecast')

In [22]:
#Próximos 12 meses
plot_model(final, plot='forecast', data_kwargs={'fh' : 12})